# Análisis inicial

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np 
import geopandas 
import matplotlib.pyplot as plt           
import pandas as pd     
import json 
import requests
import overpy
from tqdm import tqdm
from multiprocessing.dummy import Pool

In [ ]:
Cali = geopandas.read_file('databases_and_shapefiles/Barrios Cali.zip')
Cali = Cali.to_crs(epsg=4326)

In [ ]:
Cali

In [ ]:
Cali.plot()

In [ ]:
fig, ax = plt.subplots(1, 1,figsize=(10, 10))
Cali.plot(column='ESTRATO_MO', ax=ax, legend=True, legend_kwds={'label': "Estrato por Barrio",
                                                                'orientation': "horizontal"})

In [ ]:
Covid_cali = pd.read_excel('databases_and_shapefiles/Datos de covid georeferenciados en Cali para 28.06.2021.xlsx',
                           sheet_name='Datos no faltantes')
Covid_cali

In [ ]:
Covid_cali = geopandas.GeoDataFrame(Covid_cali, geometry=geopandas.points_from_xy(Covid_cali.Longitud, 
                                                                                  Covid_cali.Latitud))
Covid_cali = Covid_cali.set_crs(epsg=4326, inplace=True, allow_override=True)
Covid_cali

In [ ]:
Covid19_Cali = geopandas.sjoin(Covid_cali, Cali, how="left", op="within")
Covid19_Cali1 = Covid19_Cali[Covid19_Cali['index_right'].isnull()==False]

In [ ]:
Covid_Cali_Cantidad = Covid19_Cali1.groupby('NOMBRE').agg({'index_right':'count'})
Covid_Cali_Cantidad

In [ ]:
Covid_x_barrio = Cali.merge(Covid_Cali_Cantidad, left_on='NOMBRE', right_on=Covid_Cali_Cantidad.index,how='left')
Covid_x_barrio = Covid_x_barrio[['geometry','index_right','NOMBRE']]
fig, ax = plt.subplots(1, 1,figsize=(10, 10))

Covid_x_barrio.plot(column='index_right',ax=ax,legend=True, legend_kwds={'label': "Covid-19 por barrio",
                                                                         'orientation': "horizontal"})

In [ ]:
Covid_Cali_Cantidad.sort_values(by='index_right', ascending=False)

In [ ]:
Covid_Cali_Cantidad.sum() 

In [ ]:
189588 - Covid_Cali_Cantidad.sum() # 2.676 casos que son fuera del area metropolitana de Cali

In [ ]:
## Datos que no están en el polígono de Cali:
Covid_noCali = geopandas.sjoin(Covid_cali, Cali, how="left", op="within")
Covid_noCali

In [ ]:
Covid_noCali2 = Covid_noCali[Covid_noCali.isnull().any(1)]
Covid_noCali2

In [ ]:
Socioeconomicos_cali = pd.read_excel('databases_and_shapefiles/Base_final.xlsx', sheet_name='Base_final')
Socioeconomicos_cali

In [ ]:
Covid_x_barrio

In [ ]:
Econ_Cali = pd.merge(Socioeconomicos_cali, Covid_x_barrio, on="NOMBRE")
Econ_Cali

# Inicia el Análisis exploratorio

In [ ]:
Datos_econ_cali = pd.read_excel('databases_and_shapefiles/Base_final.xlsx')
Datos_econ_cali

In [ ]:
import seaborn as sns
Datos_econ_cali.columns[5:227]

In [ ]:
Datos_econ_cali.iloc[:,np.r_[5:227,229]]

In [ ]:
g = sns.pairplot(Datos_econ_cali.iloc[:,5:7], plot_kws = {"color": "darkblue"},
                 diag_kws = {"color":"darkblue"})
g.fig.suptitle("Relación entre pares de variables",
               fontsize = 26,
               fontweight = "bold")
plt.subplots_adjust(top=0.9)
plt.show()

In [ ]:
#sns.color_palette("tab10")
plot_kws={"s": 3}
g = sns.pairplot(Datos_econ_cali[["Poblacion_total", "Adulto_mayor", "ind_cobert_serv_bajo", "Comorbilidad", 
                                  "razon_letalidad", "Proporcion_trabajando","Estrato_moda","Covid"]], 
                 plot_kws = {"color": "darkblue"},
                 diag_kws = {"color":"darkblue"}, hue="Estrato_moda", palette='Dark2')#'Accent'
g.fig.suptitle("Relación entre pares de variables",
               fontsize = 26,
               fontweight = "bold")
plt.subplots_adjust(top=0.9)
plt.show()

In [ ]:
correlation_mat = Datos_econ_cali.iloc[:,np.r_[5:227,229]].corr()
corr_pairs = correlation_mat.unstack()
sorted_pairs = corr_pairs.sort_values(kind="quicksort")
print(sorted_pairs)

In [ ]:
sorted_pairs['Covid'][150:223] # Variables que presentan mayor correlación lineal con Covid

In [ ]:
g = sns.jointplot(x = "Poblacion_total", y = "Covid", data = Datos_econ_cali)
g.fig.suptitle("Población vs Covid", fontsize = 16, fontweight = "bold")
plt.subplots_adjust(top=0.9)
#g.fig.text(1,-.02, "Elaboración:", fontsize = 10, fontweight = "bold", ha = "right")
#g.fig.text(1,-.07, "Saenz.\n(2021)", fontsize = 8, ha = "right")
plt.show()

In [ ]:
for i in Datos_econ_cali.columns[5:227]:
    var = i
    data = pd.concat([Datos_econ_cali['Covid'], Datos_econ_cali[var]], axis=1)
    g = sns.jointplot(x = var, y = "Covid", data = data)
    g.fig.suptitle(i, fontsize = 16, fontweight = "bold")
    plt.subplots_adjust(top=0.9)
    g.fig.text(1,-.02, "Elaboración:", fontsize = 10, fontweight = "bold", ha = "right")
    g.fig.text(1,-.07, "Saenz.\n(2021)", fontsize = 8, ha = "right")
    plt.show()

In [ ]:
for i in Datos_econ_cali.columns[5:7]:
    var2 = i
    data2 = pd.concat([Datos_econ_cali['Covid'], Datos_econ_cali[var2]], axis=1)
    g = sns.relplot(x = var2, y = "Covid", kind = "line", data = data2)
    g.fig.suptitle(i, fontsize = 16, fontweight = "bold")
    plt.subplots_adjust(top=0.85)
    g.fig.text(1,-.02, "Elaboración:", fontsize = 10, fontweight = "bold", ha = "right")
    g.fig.text(1,-.08, "Saenz.\n(2021)", fontsize = 8, ha = "right")
    plt.show()

In [ ]:
#sns.set_style("white")
#sns.set_style('whitegrid')
for i in Datos_econ_cali.columns[5:227]:
    var = i
    data = pd.concat([Datos_econ_cali['Covid'], Datos_econ_cali[var]], axis=1)
    g = sns.jointplot(x = var, y = "Covid", data = data, kind = "reg")
    g.fig.suptitle(i, fontsize = 16, fontweight = "bold")
    plt.subplots_adjust(top=0.9)
    g.fig.text(1,-.02, "Elaboración:", fontsize = 10, fontweight = "bold", ha = "right")
    g.fig.text(1,-.07, "Saenz.\n(2021)", fontsize = 8, ha = "right")
    plt.show()

In [ ]:
g = sns.catplot(x = "Estrato_moda", y = "Covid", height = 4.5, aspect = 3, data = Datos_econ_cali)
g.fig.suptitle("Covid por barrio y estrato", fontsize = 18, fontweight = "bold")
for ax in g.axes.flat:
    ax.set_xticklabels(ax.get_xticklabels(), fontsize=9)
plt.subplots_adjust(top=0.8)
g.fig.text(1,-.02, "Elaboración:", fontsize = 12, fontweight = "bold", ha = "right")
g.fig.text(1,-.1, "Saenz (2021)", fontsize = 10, ha = "right")
plt.show()

In [ ]:
Cali = geopandas.read_file('databases_and_shapefiles/Barrios Cali.zip')
Cali = Cali.to_crs(epsg=4326)
Mapas_VarCali = Cali.merge(Datos_econ_cali, left_on='NOMBRE', right_on=Datos_econ_cali['NOMBRE'],how='left')
Mapas_VarCali = geopandas.GeoDataFrame(Mapas_VarCali, geometry='geometry_x')
#Mapas_VarCali = Mapas_VarCali[['geometry_x','Estrato_moda','NOMBRE']]
#fig, ax = plt.subplots(1, 1,figsize=(10, 10))
#Mapas_VarCali.plot(column="Estrato_moda",ax=ax,legend=True, legend_kwds={'label': "Covid-19 por barrio",'orientation': "horizontal"})

In [ ]:
fig, ax = plt.subplots(1, 1,figsize=(10, 10))
Mapas_VarCali.plot(column='Estrato_moda', ax=ax, legend=True, legend_kwds={'label': "Estrato por Barrio",'orientation': "horizontal"})

In [ ]:
Mapas_VarCali.columns[np.r_[11:234,236]]

In [ ]:
for i in Mapas_VarCali.columns[np.r_[11:234,236]]: #138
    var2 = i
    data2 = pd.concat([Mapas_VarCali['Covid'], Mapas_VarCali[var2]], axis=1)
    fig, ax = plt.subplots(1, 1,figsize=(10, 10))
    Mapas_VarCali.plot(column=var2, ax=ax, legend=True, legend_kwds={'label': var2,'orientation': "horizontal"})

In [ ]:
## Variables categóricas 
# Estrato moda 
pal = {1.0:"brown", 2.0:"red", 3.0: "orange", 4.0: "green", 5.0: "yellow", 6.0: "blue"}
g = sns.pairplot(Datos_econ_cali, vars = ['Proporcion_trabajando', 'Covid'], hue = 'Estrato_moda', palette = pal, height=5) #, hue_order = ["1", "2", "3", "4", "5", "6"]
#g.fig.suptitle("Relación entre pares de variables", fontsize = 20, ontweight = "bold")
plt.subplots_adjust(top=0.9)
#g.fig.text(1,-.02, "Elaboración:", fontsize = 12, fontweight = "bold", ha = "right")
#g.fig.text(1,-.055, "Saenz (2021)", fontsize = 10, ha = "right")
plt.show()

In [ ]:
for i in Datos_econ_cali.columns[np.r_[5:227,229]]:
    g = sns.pairplot(Datos_econ_cali, vars = [i, 'Covid'], hue = 'Estrato_moda', palette = pal, height=5) #, hue_order = ["1", "2", "3", "4", "5", "6"]
    plt.subplots_adjust(top=0.9)
    g.fig.suptitle(i, fontsize = 20)
    g.fig.text(1,-.02, "Elaboración:", fontsize = 12, fontweight = "bold", ha = "right")
    g.fig.text(1,-.055, "Saenz (2021)", fontsize = 10, ha = "right")
    plt.show()

In [ ]:
Datos_econ_cali.Estrato_moda.value_counts()

In [ ]:
g = sns.catplot(x = "Estrato_moda", kind = "count", data = Datos_econ_cali)
g.fig.suptitle("Número de barrios por estrato", fontsize = 16, fontweight = "bold")
for ax in g.axes.flat:
    ax.set_xticklabels(ax.get_xticklabels(), fontsize=9)
plt.subplots_adjust(top=0.85)
g.fig.text(1,-.02, "Elaboración:", fontsize = 10, fontweight = "bold", ha = "right")
g.fig.text(1,-.08, "Saenz (2021)", fontsize = 8, ha = "right")
plt.show()

In [ ]:
g = sns.catplot(x = "Estrato_moda", y = "Proporcion_trabajando", kind = "bar", height = 4.5, aspect = 3, data = Datos_econ_cali)
g.fig.suptitle("Proporción trabajando por estrato", fontsize = 16, fontweight = "bold")
for ax in g.axes.flat:
    ax.set_xticklabels(ax.get_xticklabels(), fontsize=9)
plt.subplots_adjust(top=0.85)
#g.fig.text(1,-.02, "Elaboración:", fontsize = 10, fontweight = "bold", ha = "right")
#g.fig.text(1,-.09, "Saenz (2021)", fontsize = 8, ha = "right")
plt.show()

In [ ]:
fig = plt.figure()
ax = sns.distplot(Datos_econ_cali["Covid"]) #.dropna()
fig.axes.append(ax)
fig.suptitle("Distribución de los casos positivos de Covid-19", fontsize = 14, fontweight = "bold")
plt.subplots_adjust(top = 0.85)
#fig.text(1,-.02, "Elaboración:", fontsize = 10, fontweight = "bold", ha = "right")
#fig.text(1,-.1, "Saenz (2021)", fontsize = 8, ha = "right")
plt.show()

In [ ]:
for i in Datos_econ_cali.columns[np.r_[5:227,229]]:
    fig = plt.figure()
    ax = sns.distplot(Datos_econ_cali[i].dropna()) #.dropna()
    fig.axes.append(ax)
    fig.suptitle(i, fontsize = 14, fontweight = "bold")
    plt.subplots_adjust(top = 0.85)
    fig.text(1,-.02, "Elaboración:", fontsize = 10, fontweight = "bold", ha = "right")
    fig.text(1,-.1, "Saenz (2021)", fontsize = 8, ha = "right")
    plt.show()

## Indices de Moran

In [ ]:
## Indice de moran
#!pip install esda
#!pip install splot
from esda.moran import Moran_BV, Moran_Local_BV
from splot.esda import plot_moran_bv_simulation, plot_moran_bv
from libpysal.weights.contiguity import Queen
from splot.esda import moran_scatterplot
from splot.esda import plot_local_autocorrelation
from esda.moran import Moran
from esda.moran import Moran_Local

### Moran univariado

In [ ]:
Datos_econ_cali = pd.read_excel('databases_and_shapefiles/Base_final.xlsx')
Cali = geopandas.read_file('databases_and_shapefiles/Barrios Cali.zip')
Cali = Cali.to_crs(epsg=4326)
base_moran = Cali.merge(Datos_econ_cali[['Covid', 'Adulto_mayor', 'ind_cobert_serv_bajo', 'Comorbilidad',
                                         'Proporcion_trabajando',
                                         'Estrato_moda']], left_on='NOMBRE', right_on=Datos_econ_cali['NOMBRE'],
                        how='left')
base_moran = geopandas.GeoDataFrame(base_moran, geometry='geometry')
base_moran.drop(['ID_BARRIO', 'COD_BARRIO','COD_COMUNA', 'NOMBRE', 'ESTRATO_MO'], axis=1, inplace=True)
base_moran = base_moran.dropna()
base_moran.plot()

In [ ]:
w = Queen.from_dataframe(geopandas.GeoDataFrame(Cali.merge(Datos_econ_cali['Covid'],
                                                           left_on='NOMBRE', right_on=Datos_econ_cali['NOMBRE'],
                                                           how='left'), geometry='geometry').dropna())
moran = Moran(Datos_econ_cali['Covid'], w)
print(moran.I)
print(moran.p_sim)

In [ ]:
#plt.rcParams["figure.figsize"] = (15,10)
moran_loc = Moran_Local(geopandas.GeoDataFrame(Cali.merge(Datos_econ_cali['Covid'], left_on='NOMBRE',
                                                          right_on=Datos_econ_cali['NOMBRE'], how='left'),
                                               geometry='geometry').dropna().Covid, 
                        Queen.from_dataframe(geopandas.GeoDataFrame(Cali.merge(Datos_econ_cali['Covid'],
                                                                               left_on='NOMBRE', 
                                                                               right_on=Datos_econ_cali['NOMBRE'],
                                                                               how='left'), 
                                                                    geometry='geometry').dropna()))

plot_local_autocorrelation(moran_loc, geopandas.GeoDataFrame(Cali.merge(Datos_econ_cali['Covid'], left_on='NOMBRE',
                                                                        right_on=Datos_econ_cali['NOMBRE'],
                                                                        how='left'),
                                                             geometry='geometry').dropna(), 'Covid',figsize=(18,7))
plt.show()

In [ ]:
plot_local_autocorrelation(moran_loc, geopandas.GeoDataFrame(Cali.merge(Datos_econ_cali['Covid'], left_on='NOMBRE', 
                                                                        right_on=Datos_econ_cali['NOMBRE'],how='left'),
                                                             geometry='geometry').dropna(), 'Covid', quadrant=1)
plt.show()

### Moran Bivariado

#### Adultos Mayores

In [ ]:
#Covid Adulto_mayor ind_cobert_serv_bajo Comorbilidad Proporcion_trabajando%in%Estrato_moda

In [ ]:
from esda.moran import Moran_Local_BV
moran_adultomay = geopandas.GeoDataFrame(Cali.merge(Datos_econ_cali[['Adulto_mayor','Covid']], left_on='NOMBRE',
                                                    right_on=Datos_econ_cali['NOMBRE'],how='left'),
                                         geometry='geometry').dropna()
w = Queen.from_dataframe(moran_adultomay)

moran_loc_bv = Moran_Local_BV(x = moran_adultomay.Adulto_mayor, y = moran_adultomay.Covid, w = w)
fig, ax = moran_scatterplot(moran_loc_bv, p=0.05)
fig.set_size_inches(15.5, 10.5)
ax.set_xlabel('Covid')
ax.set_ylabel('Rezago espacial Adultos mayores')
plt.show()

In [ ]:
plot_local_autocorrelation(moran_loc_bv, moran_adultomay, 'Adulto_mayor',figsize=(18,7))
plt.show()

In [ ]:
plot_local_autocorrelation(moran_loc_bv, moran_adultomay, 'Adulto_mayor', quadrant=1, figsize=(18,7))
plt.show()

#### ind_cobert_serv_bajo

In [ ]:
moran_ind_cobertura = geopandas.GeoDataFrame(Cali.merge(Datos_econ_cali[['ind_cobert_serv_bajo','Covid']], left_on='NOMBRE',
right_on=Datos_econ_cali['NOMBRE'],how='left'), geometry='geometry').dropna()
w2 = Queen.from_dataframe(moran_ind_cobertura)

moran_loc_bv2 = Moran_Local_BV(x = moran_ind_cobertura.ind_cobert_serv_bajo, y = moran_ind_cobertura.Covid, w = w2)
fig, ax = moran_scatterplot(moran_loc_bv2, p=0.05)
fig.set_size_inches(15.5, 10.5)
ax.set_xlabel('Covid')
ax.set_ylabel('Rezago espacial ind_cobert_serv_bajo')
plt.show()

In [ ]:
plot_local_autocorrelation(moran_loc_bv2, moran_ind_cobertura, 'ind_cobert_serv_bajo',figsize=(18,7))
plt.show()

In [ ]:
plot_local_autocorrelation(moran_loc_bv2, moran_ind_cobertura, 'ind_cobert_serv_bajo', quadrant=1, figsize=(18,7))
plt.show()

#### Comorbilidad

In [ ]:
moran_Comorbilidad = geopandas.GeoDataFrame(Cali.merge(Datos_econ_cali[['Comorbilidad','Covid']], left_on='NOMBRE',
right_on=Datos_econ_cali['NOMBRE'],how='left'), geometry='geometry').dropna()
w3 = Queen.from_dataframe(moran_Comorbilidad)

moran_loc_bv3 = Moran_Local_BV(x = moran_Comorbilidad.Comorbilidad, y = moran_Comorbilidad.Covid, w = w3)
fig, ax = moran_scatterplot(moran_loc_bv3, p=0.05)
fig.set_size_inches(15.5, 10.5)
ax.set_xlabel('Covid')
ax.set_ylabel('Rezago Comorbilidad')
plt.show()

In [ ]:
plot_local_autocorrelation(moran_loc_bv3, moran_Comorbilidad, 'Comorbilidad', figsize=(18,7))
plt.show()

In [ ]:
plot_local_autocorrelation(moran_loc_bv3, moran_Comorbilidad, 'Comorbilidad', quadrant=1, figsize=(18,7))
plt.show()

# Spatial cross-validation

In [ ]:
#https://towardsdatascience.com/spatial-cross-validation-using-scikit-learn-74cb8ffe0ab9
%matplotlib inline
import pandas as pd
import numpy as np 
import geopandas 
import matplotlib.pyplot as plt           
import pandas as pd     
import json 
import requests
import overpy
from tqdm import tqdm
from multiprocessing.dummy import Pool
Cali = geopandas.read_file('databases_and_shapefiles/Barrios Cali.zip')
Cali = Cali.to_crs(epsg=4326)
Cali

In [ ]:
Cali.plot()

In [ ]:
Covid_cali = pd.read_excel('databases_and_shapefiles/Cali.xlsx', sheet_name='Sheet1')
Covid_cali

In [ ]:
Cali = Cali.merge(Covid_cali, on='ID_BARRIO')#.drop_duplicates(subset=['NAME'])
Cali = Cali.drop(['COD_BARRIO', 'COD_COMUNA', 'NOMBRE', 'ESTRATO_MO', 'Centroide_X', 'Centroide_Y'], axis=1)
Cali

In [ ]:
# centroids
Cali['x_coord'] = Cali.geometry.centroid.x
Cali['y_coord'] = Cali.geometry.centroid.y
# Gráfico de casos de covid-19
fig, ax = plt.subplots(dpi=120)
Cali.plot(ax=ax,column='Covid',legend=True,figsize=(4,2),cmap='viridis')
plt.title("Map of The Dependent Variable: \nCovid - 19 ",fontsize=8)
ax.axis('off')

In [ ]:
# Proporción de personas trabajando por estrato. Después de eso, eliminar las variables separadas
Cali['Prop_trabaj_estrato'] = ((Cali['Proporcion_trabajando'] - min(Cali['Proporcion_trabajando']))/(max(Cali['Proporcion_trabajando']) - min(Cali['Proporcion_trabajando'])))*Cali['Estrato_moda']
Cali = Cali.drop(['Proporcion_trabajando', 'Estrato_moda'], axis=1)
Cali = Cali.dropna().reset_index(drop=True)
Cali.info()

In [ ]:
X_vars = ['Adulto_mayor','ind_cobert_serv_bajo','Comorbilidad','razon_letalidad','Prop_trabaj_estrato','x_coord','y_coord']
X = Cali[X_vars]
y = Cali['Covid']
"""
## Red neuronal 1
#https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
# Use scikit-learn to grid search the number of neurons
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
#from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
from keras.constraints import maxnorm
from sklearn.model_selection import GridSearchCV
# Function to create model, required for KerasClassifier

def create_model(optimizer='adam',learn_rate=0.01, momentum=0,init_mode='normal', activation='relu',dropout_rate=0.0, 
                 weight_constraint=0,neurons=1):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_dim=X.shape[1], kernel_initializer=init_mode, activation=activation, 
                    kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons, input_dim=X.shape[1], kernel_initializer=init_mode, activation=activation, 
                    kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons, input_dim=X.shape[1], kernel_initializer=init_mode, activation=activation, 
                    kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, kernel_initializer=init_mode, activation='linear'))
    # Compile model
    model.compile(loss='mean_absolute_error', optimizer=optimizer, metrics=['mean_absolute_error'])
    return model

import numpy
seed = 7
numpy.random.seed(seed)

model = KerasRegressor(build_fn=create_model, verbose=0)

# define the grid search parameters
batch_size = [10, 20, 30] #, 40, 50, 60, 70, 80, 90, 100
epochs = [5, 7, 10, 15] #, 30, 40, 50, 60, 70, 80, 90, 100, 20
optimizer = ['SGD', 'RMSprop', 'Adam'] #  'Adagrad', 'Adadelta'
learn_rate = [0.0001, 0.001, 0.01, 0.03] #, 0.1, 0.05
momentum = [0.0, 0.05, 0.1] #, 0.4, 0.6, 0.8, 0.9 , 0.2, 0.3]
init_mode = ['normal'] #'lecun_uniform', 'zero', 'glorot_normal', 'glorot_uniform', 'normal', 'he_normal', 'he_uniform'
activation = ['relu', 'tanh', 'linear'] #'softmax', 'softplus', 'softsign', 'hard_sigmoid',  'sigmoid',
weight_constraint = [1, 2] #3,4,5
dropout_rate = [0.01, 0.02, 0.05, 0.1, 0.2] #, 0.3, 0.4
neurons = [5, 10, 15, 20, 25] #, 30, 40, 50

param_grid = dict(batch_size=batch_size, epochs=epochs,optimizer=optimizer,learn_rate=learn_rate, momentum=momentum, 
                  init_mode=init_mode, activation=activation, 
                  weight_constraint=weight_constraint, dropout_rate=dropout_rate, neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
"""

## Red Neuronal

In [ ]:
# Modelo de red neuronal
#X_vars = ['Adulto_mayor','ind_cobert_serv_bajo','Comorbilidad','razon_letalidad','Prop_trabaj_estrato']
#X = Cali[X_vars]
#y = Cali['Covid']

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
#from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
from keras.constraints import maxnorm
from sklearn.model_selection import GridSearchCV
# Function to create model, required for KerasClassifier

# create model
def create_model(optimizer='adam',learn_rate=0.01, momentum=0,init_mode='normal', activation='relu',dropout_rate=0.0, 
                 weight_constraint=0,neurons=1):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_dim=X.shape[1], kernel_initializer=init_mode, activation=activation, 
                    kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons, input_dim=X.shape[1], kernel_initializer=init_mode, activation=activation, 
                    kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons, input_dim=X.shape[1], kernel_initializer=init_mode, activation=activation, 
                    kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, kernel_initializer=init_mode, activation='linear'))
    # Compile model
    model.compile(loss='mean_absolute_error', optimizer=optimizer, metrics=['mean_absolute_error'])
    return model

import numpy
seed = 7
numpy.random.seed(seed)
model = KerasRegressor(build_fn=create_model, verbose=0)

# define the grid search parameters
batch_size = [10]
epochs = [20]
optimizer = ['Adam']
learn_rate = [0.03]
momentum = [0.1]
init_mode = ['uniform']
activation = ['linear']
weight_constraint = [1]
dropout_rate = [0.06] 
neurons = [30]

param_grid = dict(batch_size=batch_size, epochs=epochs,optimizer=optimizer,learn_rate=learn_rate, momentum=momentum, 
                  init_mode=init_mode, activation=activation, 
                  weight_constraint=weight_constraint, dropout_rate=dropout_rate, neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, verbose=1) #, cv=3 no va cv porque particiona la data antes
#grid_result = grid.fit(X, y)

In [ ]:
Cali['Comuna'] = Cali['Comuna'].astype('int').astype('str')
Cali.info()

In [ ]:
from sre_parse import Verbose
from sklearn.model_selection import GroupKFold, cross_val_predict
barrios = Cali['Comuna'].values
group_kfold = GroupKFold(n_splits=len(Cali['Comuna'].unique())) 
# Generator for the train/test indices

barrio_kfold = group_kfold.split(X, y, barrios)
# Create a nested list of train and test indices for each fold
train_indices, test_indices = [list(traintest) for traintest in zip(*barrio_kfold)]
barrio_cv = [*zip(train_indices,test_indices)]
predictions = cross_val_predict(grid, X, y, cv=barrio_cv, n_jobs = 1, verbose=1)
predictions

In [ ]:
from sklearn.model_selection import cross_val_score
predictions2 = cross_val_score(grid, X, y, cv=barrio_cv, n_jobs = 1, verbose=1, scoring='neg_mean_absolute_error')
predictions3 = cross_val_score(grid, X, y, cv=barrio_cv, n_jobs = 1, verbose=1, scoring='neg_mean_squared_error')
predictions4 = cross_val_score(grid, X, y, cv=barrio_cv, n_jobs = 1, verbose=1, scoring='r2')
print("MAE: %.3f (%.3f)" % (predictions2.mean(), predictions2.std())) #A value of 0 indicates no error or perfect predictions.
print("MSE: %.3f (%.3f)" % (predictions3.mean(), predictions3.std())) #This metric too is inverted so that the results are increasing.
print("R^2: %.3f (%.3f)" % (predictions4.mean(), predictions4.std()))

# Con el modelo inicial de los parámetros de red neuronal al azar dio esto:
#MAE: -103.079 (44.947)
#MSE: -23865.132 (20660.603)
#R^2: 0.800 (0.398)

#MAE: -150.027 (69.514)
#MSE: -68707.495 (101043.422)
#R^2: 0.634 (0.386)

#MAE: -103.352 (48.740)
#MSE: -26834.109 (30055.439)
#R^2: 0.806 (0.270)

In [ ]:
print(len(predictions))
print(np.mean(predictions))

In [ ]:
print(len(y))
print(np.mean(y))

In [ ]:
for i in range(len(barrio_cv)):
    print(barrio_cv[i][1])

In [ ]:
Cali['Comuna'].unique()

In [ ]:
indices_comuna = []
for i in Cali['Comuna'].unique():
    indices_comuna.append([Cali[Cali['Comuna']==i].index, i])

In [ ]:
for i in range(len(indices_comuna)):
    print(indices_comuna[i][0].values)

In [ ]:
Cali['prediccion_nn'] = predictions
Cali

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, dpi=150)
Cali.plot(ax=ax1,column='Covid',legend=True,figsize=(2,1),cmap='viridis')
ax1.axis('off')
Cali.plot(ax=ax2,column='prediccion_nn',legend=True,figsize=(2,1),cmap='viridis')
ax2.axis('off')
plt.suptitle("Map of The real values vs predicted-nn values of Covid - 19")

# Unir datos espaciales de la segunda base de datos: con vacunas

In [ ]:
import pandas as pd
import numpy as np 
import geopandas 
import matplotlib.pyplot as plt           
import pandas as pd     
import json
import requests
import overpy
from tqdm import tqdm
from multiprocessing.dummy import Pool

In [ ]:
base_vacunados = pd.read_excel('databases_and_shapefiles/Casos positivos acumulados a Mayo 12 2022.xlsx',
                               sheet_name='Hoja1')
print(base_vacunados.shape)
base_vacunados.head()

In [ ]:
datos_base_anterior = pd.read_excel('databases_and_shapefiles/Cali.xlsx', sheet_name='Sheet1')
print(datos_base_anterior.shape)
datos_base_anterior.head()

In [ ]:
array = datos_base_anterior['Barrio_Urbanizacion_o_Sector'].values
base_vacunados2 = base_vacunados.loc[base_vacunados['barrio'].isin(array)]
print(base_vacunados2.shape)
base_vacunados2.head()

In [ ]:
base_vacunados_index = base_vacunados.index
base_vacunados2_index = base_vacunados2.index
mask = ~base_vacunados_index.isin(base_vacunados2_index)
base_vacunados3 = base_vacunados.loc[mask]
base_vacunados3

In [ ]:
base_vacunados2.reset_index(drop=True, inplace=True)
base_vacunados3.reset_index(drop=True, inplace=True)

# Agregar información del mapa shape file
Cali = geopandas.read_file('databases_and_shapefiles/Barrios Cali.zip')
Cali = Cali.to_crs(epsg=4326)
Cali.plot()

In [ ]:
Cali['COD_COMUNA'] = Cali['COD_COMUNA'].astype(np.int64)#
Cali['ID_BARRIO'] = Cali['ID_BARRIO'].astype(np.int64)#

datos_base_anterior = datos_base_anterior[datos_base_anterior['Comuna'].notna()]
datos_base_anterior['Comuna'] = datos_base_anterior['Comuna'].astype(np.int64)#
datos_base_anterior['ID_BARRIO'] = datos_base_anterior['ID_BARRIO'].astype(np.int64)#
datos_base_anterior[['ID_BARRIO', 'Comuna']].info()

Cali = pd.merge(Cali, datos_base_anterior,  how='left', left_on=['ID_BARRIO', 'COD_COMUNA'], right_on = ['ID_BARRIO', 'Comuna'])
Cali

In [ ]:
Cali.loc[Cali['NOMBRE']=='Silo�', 'Barrio_Urbanizacion_o_Sector'] = 'Siloe'
Cali.loc[Cali['NOMBRE']=='Urb. Tequendama', 'Barrio_Urbanizacion_o_Sector'] = 'Urbanización Tequendama'
Cali.loc[Cali['NOMBRE']=='Villamercedes I - Villa Luz - Las G', 'Barrio_Urbanizacion_o_Sector'] = 'Villamercedes I-Villa Luz-Las Garzas'
Cali.loc[Cali['NOMBRE']=='Caldas', 'Barrio_Urbanizacion_o_Sector'] = 'Barrio Caldas'
Cali.loc[Cali['NOMBRE']=='Las Quintas de Don Simon', 'Barrio_Urbanizacion_o_Sector'] = 'Las Quintas de Don Simón'
Cali.loc[Cali['NOMBRE']=='Urb. Ciudad Jardin', 'Barrio_Urbanizacion_o_Sector'] = 'Urbanizacion Ciudad Jardin'
Cali[Cali['NOMBRE']=='Urb. Ciudad Jardin']

In [ ]:
Cali['Barrio_Urbanizacion_o_Sector'].isna().sum()

In [ ]:
base_vacunados3 = base_vacunados3[(base_vacunados3['Latitud - X']!='sin dato') & (base_vacunados3['Longitud Y']!='sin dato')]
base_vacunados3['Latitud - X'] = base_vacunados3['Latitud - X'].astype(float)#.apply(lambda x: x)
base_vacunados3['Longitud Y'] = base_vacunados3['Longitud Y'].astype(float)#
base_vacunados3[['Longitud Y','Latitud - X']].info()

In [ ]:
base_vacunados3 = geopandas.GeoDataFrame(base_vacunados3, geometry = geopandas.points_from_xy(base_vacunados3['Longitud Y'], base_vacunados3['Latitud - X']))
base_vacunados3 = base_vacunados3.set_crs(epsg=4326, inplace=True, allow_override=True)
base_vacunados3.head(3)

In [ ]:
base_vacunados4 = geopandas.sjoin(base_vacunados3, Cali, how="left", op="within")
base_vacunados4 = base_vacunados4[base_vacunados4['Barrio_Urbanizacion_o_Sector'].isnull()==False]
base_vacunados4.reset_index(inplace=True, drop=True)
print(base_vacunados4.shape)
base_vacunados4.head(3)

In [ ]:
base_vacunados4.columns

In [ ]:
base_vacunados4 = base_vacunados4.drop(['barrio', 'geometry', 'index_right', 'ID_BARRIO', 'COD_BARRIO', 'COD_COMUNA', 'NOMBRE','ESTRATO_MO',
                                        'Comuna_right', 'Poblacion_total', 'Adulto_mayor', 'ind_cobert_serv_bajo', 'Comorbilidad', 'razon_letalidad', 'Proporcion_trabajando',
                                        'Estrato_moda', 'Covid', 'Centroide_X', 'Centroide_Y'], axis=1)
base_vacunados4.rename(columns = {'Barrio_Urbanizacion_o_Sector':'barrio', 'Comuna_left':'Comuna'}, inplace = True)
print(base_vacunados4.shape)
base_vacunados4.head(3)

In [ ]:
print(base_vacunados2.shape)
base_vacunados2.head(3)

In [ ]:
base_final = pd.concat([base_vacunados2, base_vacunados4])
base_final['unos'] = 1
print(base_final.shape)
base_final.head(3)

In [ ]:
base_final.columns

In [ ]:
base_final['No_pruebas'].value_counts()

In [ ]:
base_final['ocup_sitrep'].value_counts()

In [ ]:
base_final['profesion otr fuente'].value_counts()

In [ ]:
base_final['ProfesiónoOcupación'].value_counts()

In [ ]:
base_final['tip_ss_'].value_counts()

In [ ]:
base_final.loc[base_final['tip_ss_'] == 'C', 'tip_ss_'] = 'Contributivo'
base_final.loc[base_final['tip_ss_'] == 'c', 'tip_ss_'] = 'Contributivo'
base_final.loc[base_final['tip_ss_'] == 'sin dato', 'tip_ss_'] = np.nan
base_final.loc[base_final['tip_ss_'] == 0, 'tip_ss_'] = np.nan
base_final.loc[base_final['tip_ss_'] == 'S', 'tip_ss_'] = 'Subsidiado'
base_final.loc[base_final['tip_ss_'] == 's', 'tip_ss_'] = 'Subsidiado'
base_final.loc[base_final['tip_ss_'] == 'P', 'tip_ss_'] = 'Excepción'

base_final.loc[base_final['tip_ss_'] == 'p', 'tip_ss_'] = 'Excepción'
base_final.loc[base_final['tip_ss_'] == 'N', 'tip_ss_'] = 'No Asegurado'
base_final.loc[base_final['tip_ss_'] == 'n', 'tip_ss_'] = 'No Asegurado'
base_final.loc[base_final['tip_ss_'] == 'I', 'tip_ss_'] = 'Indeterminado/ pendiente'
base_final.loc[base_final['tip_ss_'] == 'I', 'tip_ss_'] = 'Indeterminado/ pendiente'
base_final.loc[base_final['tip_ss_'] == 'E', 'tip_ss_'] = 'Especial'
base_final.loc[base_final['tip_ss_'] == 'e', 'tip_ss_'] = 'Especial'
base_final['tip_ss_'].value_counts()

In [ ]:
base_final['sintomatico'].value_counts()

In [ ]:
base_final['nom_vac'].value_counts()

In [ ]:
base_final['ultima_dos'].value_counts()

In [ ]:
base_final['ultima_dos2'] = ""
base_final.loc[base_final['ultima_dos'] == '2 = segunda dosis', 'ultima_dos2'] = 2
base_final.loc[base_final['ultima_dos'] == '4 = dosis refuerzo', 'ultima_dos2'] = 4
base_final.loc[base_final['ultima_dos'] == 'sin dato', 'ultima_dos2'] = np.nan
base_final.loc[base_final['ultima_dos'] == '1 = primera dosis', 'ultima_dos2'] = 1
base_final.loc[base_final['ultima_dos'] == '3 = unica', 'ultima_dos2'] = 3
base_final.loc[base_final['ultima_dos'] == '5 = cuarta dosis', 'ultima_dos2'] = 5
base_final['ultima_dos2'].value_counts()

In [ ]:
# Variables importantes para agregar por barrio:
# 'ProfesiónoOcupación', 
# 'profesion otr fuente', 
# 'ocup_sitrep',
# tip_ss_,                                                                  Ya
# 'No_pruebas',                                                             Ya
# 'sintomatico', 'tos', 'fiebre', 'odinofagia', 'dif_res', 'adinamia',      Ya
# 'ultima_dos',                                                             Ya
# 'nom_vac'                                                                 Ya

In [ ]:
# ahora agregar ultima_dos por barrio
base_final_cantidad = base_final[base_final['ultima_dos2'].notna()].groupby('barrio').agg({'ultima_dos2':'count'})

dosis_vacuna_x_barrio = Cali.merge(base_final_cantidad, left_on = 'Barrio_Urbanizacion_o_Sector', right_on = base_final_cantidad.index, how='left')
dosis_vacuna_x_barrio = dosis_vacuna_x_barrio[['geometry','ultima_dos2','Barrio_Urbanizacion_o_Sector']]

fig, ax = plt.subplots(1, 1,figsize=(10, 10))
dosis_vacuna_x_barrio.plot(column='ultima_dos2', ax=ax, legend=True, legend_kwds={'label': "Dosis de vacunas aplicadas por barrio", 'orientation': "horizontal"})

In [ ]:
# ahora agregar No_pruebas por barrio
base_final_cantidad2 = base_final[base_final['No_pruebas'].notna()].groupby('barrio').agg({'No_pruebas':'count'})

N_pruebas_x_barrio = Cali.merge(base_final_cantidad2, left_on = 'Barrio_Urbanizacion_o_Sector', right_on = base_final_cantidad2.index, how='left')
N_pruebas_x_barrio = N_pruebas_x_barrio[['geometry','No_pruebas','Barrio_Urbanizacion_o_Sector']]

fig, ax = plt.subplots(1, 1,figsize=(10, 10))
N_pruebas_x_barrio.plot(column='No_pruebas', ax=ax, legend=True, legend_kwds={'label': "N° pruebas aplicadas por barrio", 'orientation': "horizontal"})

In [ ]:
# ahora agregar nom_vac por barrio
base_final_cantidad3 = base_final[base_final['nom_vac']!='sin dato'][['barrio',
                                                                      'nom_vac','unos']].pivot_table(index='barrio', columns='nom_vac', 
                                                                                                     values='unos', aggfunc=np.sum, fill_value=0)

Nom_vac_x_barrio = Cali.merge(base_final_cantidad3, left_on = 'Barrio_Urbanizacion_o_Sector', right_on = base_final_cantidad3.index, how='left')
Nom_vac_x_barrio = Nom_vac_x_barrio[['geometry','Pfizer','Barrio_Urbanizacion_o_Sector']]

fig, ax = plt.subplots(1, 1,figsize=(10, 10))
Nom_vac_x_barrio.plot(column='Pfizer', ax=ax, legend=True, legend_kwds={'label': "N° vacunas Pfizer aplicadas por barrio", 'orientation': "horizontal"})

In [ ]:
base_final_cantidad3[base_final_cantidad3.index == 'Siloe']

In [ ]:
base_final[base_final['nom_vac']!='sin dato'][['barrio', 'nom_vac']][base_final[base_final['nom_vac']!='sin dato'][['barrio', 'nom_vac']]['barrio']=='Siloe']['nom_vac'].value_counts()

In [ ]:
# ahora agregar tipo de seguridad social por barrio
base_final_cantidad4 = base_final[base_final['tip_ss_']!='sin dato'][['barrio',
                                                                      'tip_ss_','unos']].pivot_table(index='barrio', columns='tip_ss_', 
                                                                                                     values='unos', aggfunc=np.sum, fill_value=0)

tip_ss_x_barrio = Cali.merge(base_final_cantidad4, left_on = 'Barrio_Urbanizacion_o_Sector', right_on = base_final_cantidad4.index, how='left')
tip_ss_x_barrio = tip_ss_x_barrio[['geometry','Subsidiado','Barrio_Urbanizacion_o_Sector']]

fig, ax = plt.subplots(1, 1,figsize=(10, 10))
tip_ss_x_barrio.plot(column='Subsidiado', ax=ax, legend=True, legend_kwds={'label': "N° de subsidiados por barrio", 'orientation': "horizontal"})

In [ ]:
base_final_cantidad4[base_final_cantidad4.index == 'Siloe']

In [ ]:
base_final[base_final['barrio']=='Siloe']['tip_ss_'].value_counts()

In [ ]:
# ahora agregar sintomático y síntomas por barrio
#'sintomatico', 'tos', 'fiebre', 'odinofagia', 'dif_res', 'adinamia'

In [ ]:
base_final_cantidad5 = base_final[base_final['sintomatico']!='sin dato'][['barrio',
                                                                          'sintomatico','unos']].pivot_table(index='barrio', columns='sintomatico', 
                                                                                                             values='unos', aggfunc=np.sum, fill_value=0)

sintomatico_x_barrio = Cali.merge(base_final_cantidad5, left_on = 'Barrio_Urbanizacion_o_Sector', right_on = base_final_cantidad5.index, how='left')
sintomatico_x_barrio = sintomatico_x_barrio[['geometry','no','Barrio_Urbanizacion_o_Sector']]

fig, ax = plt.subplots(1, 1,figsize=(10, 10))
sintomatico_x_barrio.plot(column='no', ax=ax, legend=True, legend_kwds={'label': "N° de asintomáticos por barrio", 'orientation': "horizontal"})

In [ ]:
base_final['sintomatico'].value_counts()

In [ ]:
print(np.sum(base_final_cantidad5['si']))
print(np.sum(base_final_cantidad5['no']))

In [ ]:
base_final.loc[base_final['tos']==1, 'tos'] = 'si'
base_final.loc[base_final['tos']==0, 'tos'] = 'no'

base_final.loc[base_final['fiebre']==1, 'fiebre'] = 'si'
base_final.loc[base_final['fiebre']==0, 'fiebre'] = 'no'

base_final.loc[base_final['odinofagia']==1, 'odinofagia'] = 'si'
base_final.loc[base_final['odinofagia']==0, 'odinofagia'] = 'no'

base_final.loc[base_final['dif_res']==1, 'dif_res'] = 'si'
base_final.loc[base_final['dif_res']==0, 'dif_res'] = 'no'

base_final.loc[base_final['adinamia']==1, 'adinamia'] = 'si'
base_final.loc[base_final['adinamia']==0, 'adinamia'] = 'no'

In [ ]:
base_final_cantidad6 = base_final[base_final['tos']!='sin dato'][['barrio',
                                                                  'tos','unos']].pivot_table(index='barrio', columns='tos', 
                                                                                             values='unos', aggfunc=np.sum, fill_value=0)

base_final_cantidad7 = base_final[base_final['fiebre']!='sin dato'][['barrio',
                                                                     'fiebre','unos']].pivot_table(index='barrio', columns='fiebre', 
                                                                                                   values='unos', aggfunc=np.sum, fill_value=0)

base_final_cantidad8 = base_final[base_final['odinofagia']!='sin dato'][['barrio',
                                                                         'odinofagia','unos']].pivot_table(index='barrio', columns='odinofagia', 
                                                                                                           values='unos', aggfunc=np.sum, fill_value=0)
                                                                                                        
base_final_cantidad9 = base_final[base_final['dif_res']!='sin dato'][['barrio',
                                                                      'dif_res','unos']].pivot_table(index='barrio', columns='dif_res', 
                                                                                                     values='unos', aggfunc=np.sum, fill_value=0)

base_final_cantidad10 = base_final[base_final['adinamia']!='sin dato'][['barrio',
                                                                        'adinamia','unos']].pivot_table(index='barrio', columns='adinamia', 
                                                                                                        values='unos', aggfunc=np.sum, fill_value=0)

In [ ]:
base_final_cantidad.rename(columns = {'ultima_dos2':'n_de_dosis'}, inplace = True)
base_final_cantidad2.rename(columns = {'No_pruebas':'n_de_pruebas'}, inplace = True)
base_final_cantidad4.rename(columns = {'Contributivo':'ss_contributivo',
                                       'Especial':'ss_especial', 
                                       'Excepción':'ss_excepción',
                                       'Indeterminado/ pendiente':'ss_indeterminado_pendiente',
                                       'No Asegurado':'ss_no_asegurado',
                                       'Subsidiado':'ss_subsidiado'}, inplace = True)
base_final_cantidad5.rename(columns = {'no':'sintomatico_no',
                                       'si':'sintomatico_si'}, inplace = True)
base_final_cantidad6.rename(columns = {'no':'tos_no',
                                       'si':'tos_si'}, inplace = True)
base_final_cantidad7.rename(columns = {'no':'fiebre_no',
                                       'si':'fiebre_si'}, inplace = True)
base_final_cantidad8.rename(columns = {'no':'odinofagia_no',
                                       'si':'odinofagia_si'}, inplace = True)
base_final_cantidad9.rename(columns = {'no':'dif_res_no',
                                       'si':'dif_res_si'}, inplace = True)
base_final_cantidad10.rename(columns = {'no':'adinamia_no',
                                        'si':'adinamia_si'}, inplace = True)

In [ ]:
#pd.DataFrame(base_final['barrio'].value_counts()).to_excel('covid_nueva_base.xlsx')

In [ ]:
#base_final[base_final['unos']!='sin dato'][['barrio','unos']].pivot_table(index='barrio', values='unos', aggfunc=np.sum, fill_value=0).to_excel('covid_nueva_base2.xlsx')

In [ ]:
base_final_cantidad

In [ ]:
base_final_cantidad2

In [ ]:
base_final_cantidad3

In [ ]:
base_final_cantidad4

In [ ]:
base_final_cantidad5

In [ ]:
base_final_cantidad6

In [ ]:
base_final_cantidad7

In [ ]:
base_final_cantidad8

In [ ]:
base_final_cantidad9

In [ ]:
base_final_cantidad10

In [ ]:
base_final_cantidad

In [ ]:
base_final_barrio = pd.merge(base_final_cantidad.reset_index(), base_final_cantidad2.reset_index(), left_on='barrio', right_on='barrio')
base_final_barrio = pd.merge(base_final_barrio, base_final_cantidad3.reset_index(), left_on='barrio', right_on='barrio')
base_final_barrio = pd.merge(base_final_barrio, base_final_cantidad4.reset_index(), left_on='barrio', right_on='barrio')
base_final_barrio = pd.merge(base_final_barrio, base_final_cantidad5.reset_index(), left_on='barrio', right_on='barrio')
base_final_barrio = pd.merge(base_final_barrio, base_final_cantidad6.reset_index(), left_on='barrio', right_on='barrio')
base_final_barrio = pd.merge(base_final_barrio, base_final_cantidad7.reset_index(), left_on='barrio', right_on='barrio')
base_final_barrio = pd.merge(base_final_barrio, base_final_cantidad8.reset_index(), left_on='barrio', right_on='barrio')
base_final_barrio = pd.merge(base_final_barrio, base_final_cantidad9.reset_index(), left_on='barrio', right_on='barrio')
base_final_barrio = pd.merge(base_final_barrio, base_final_cantidad10.reset_index(), left_on='barrio', right_on='barrio')
base_final_barrio

In [ ]:
datos_base_anterior = pd.read_excel('databases_and_shapefiles/Cali.xlsx', sheet_name='Sheet1')
base_final_final = pd.merge(datos_base_anterior, base_final_barrio, left_on='Barrio_Urbanizacion_o_Sector', right_on='barrio', how='left')
base_final_final

In [ ]:
#base_final_final.to_excel('Cali_nueva_base.xlsx')

# Modelos de machine learning y deep learning con base de datos anterior

## Preparación de la base de datos

In [ ]:
#!pip install ax-platform
#https://ax.dev/docs/installation.html
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from ax.service.ax_client import AxClient
from ax.utils.notebook.plotting import render, init_notebook_plotting
from sklearn.svm import SVR
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import xgboost
from sklearn.ensemble import GradientBoostingRegressor
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [ ]:
Cali = pd.read_excel('databases_and_shapefiles/Cali.xlsx', sheet_name='Sheet1')
Cali = Cali.iloc[:-1,]
Cali['Covid'] = Cali['Covid'].apply(lambda x: np.int64(x))
Cali.head(3)

In [ ]:
Cali.info()

In [ ]:
Cali['Prop_trabaj_estrato'] = ((Cali['Proporcion_trabajando'] - min(Cali['Proporcion_trabajando']))/(max(Cali['Proporcion_trabajando']) - min(Cali['Proporcion_trabajando'])))*Cali['Estrato_moda']
Cali['log_Comorbilidad'] = np.log(Cali['Comorbilidad'])
Cali = Cali.drop(['Proporcion_trabajando', 'Estrato_moda', 'Comorbilidad'], axis=1)
X_vars = ['Adulto_mayor','ind_cobert_serv_bajo','log_Comorbilidad','razon_letalidad','Prop_trabaj_estrato', 'Comuna']
X = Cali[X_vars]
y = Cali[['Covid', 'Comuna']]
X.head(3)

In [ ]:
from sklearn.impute import KNNImputer
knn = KNNImputer(n_neighbors=10)
X = pd.DataFrame(knn.fit_transform(X), columns=X.columns)
X['Adulto_mayor'] = X['Adulto_mayor'].apply(lambda x: np.int64(x))
X['ind_cobert_serv_bajo'] = X['ind_cobert_serv_bajo'].apply(lambda x: np.int64(x))
X.head(3)

In [ ]:
X.info()

In [ ]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 222)
X_test = X[X['Comuna']==2.0][['Adulto_mayor','ind_cobert_serv_bajo','log_Comorbilidad','razon_letalidad','Prop_trabaj_estrato']]
X_train = X[X['Comuna']!=2.0][['Adulto_mayor','ind_cobert_serv_bajo','log_Comorbilidad','razon_letalidad','Prop_trabaj_estrato']]
y_test = y[y['Comuna']==2.0]['Covid']
y_train = y[y['Comuna']!=2.0]['Covid']

## Random Forest

In [ ]:
rf = RandomForestRegressor(max_depth=24, min_samples_split=5, n_estimators=20, min_samples_leaf=5)
rf.fit(X_train, y_train)

In [ ]:
score_rf = rf.score(X_test, y_test)
print(score_rf)

In [ ]:
pred_rf = rf.predict(X_test)
np.mean(pred_rf)

In [ ]:
n_estimators =  [10,15,20,25,30,35,40,45,50,55,60,65,70,75,90,100,150,200]#[10,15,20,25,30,35,40,45,50,55,60,65,70,75]
max_depth = [10,15,20,21,24,25,30,40,50,60]#[10,15,20,21,24,25]
min_samples_split = [1, 2, 3, 5, 10] # 2, 3, 5,
min_samples_leaf = [1, 2, 3, 5, 10] # 2, 3, 5,
param_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,}

In [ ]:
#scoring='neg_mean_absolute_error')
#scoring='neg_mean_squared_error')
#scoring='r2'

rf = RandomForestRegressor()
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = 1, verbose = 2, scoring='r2')
rf_cv = grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_estimator_

In [ ]:
score_rf = grid_search.score(X_test, y_test)
print(score_rf)

In [ ]:
pred_rf = grid_search.predict(X_test)
np.mean(pred_rf)

In [ ]:
mean_squared_error(y_test,pred_rf)

In [ ]:
mean_absolute_error(y_test,pred_rf)

In [ ]:
r2_score(y_test,pred_rf)

## Neural Network

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
#from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
from keras.constraints import maxnorm
from sklearn.model_selection import GridSearchCV
# Function to create model, required for KerasClassifier

# create model
def create_model(optimizer='adam',learn_rate=0.01, momentum=0,init_mode='normal', activation='relu',dropout_rate=0.0, 
                 weight_constraint=0,neurons=1):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train.shape[1], kernel_initializer=init_mode, activation=activation, 
                    kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons, input_dim=X_train.shape[1], kernel_initializer=init_mode, activation=activation, 
                    kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons, input_dim=X_train.shape[1], kernel_initializer=init_mode, activation=activation, 
                    kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, kernel_initializer=init_mode, activation='linear'))
    # Compile model
    model.compile(loss='mean_absolute_error', optimizer=optimizer, metrics=['mean_absolute_error'])
    return model

import numpy
seed = 7
numpy.random.seed(seed)
model = KerasRegressor(build_fn=create_model, verbose=0)

# define the grid search parameters
batch_size = [5, 10, 15]
epochs = [10, 20, 30]
optimizer = ['Adam', 'SGD', 'RMSprop']
learn_rate = [0.01, 0.05, 0.1]
momentum = [0.05, 0.1]
init_mode = ['uniform', 'normal']
activation = ['linear', 'relu', 'tanh']
weight_constraint = [1, 2]
dropout_rate = [0.06] 
neurons = [20, 30]

param_grid = dict(batch_size=batch_size, epochs=epochs,optimizer=optimizer,learn_rate=learn_rate, momentum=momentum, 
                  init_mode=init_mode, activation=activation, 
                  weight_constraint=weight_constraint, dropout_rate=dropout_rate, neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, verbose=1, cv=5) #, cv=3 no va cv porque particiona la data antes
grid_result = grid.fit(X_train, y_train)

In [ ]:
from sklearn.model_selection import cross_val_score
predictions = cross_val_score(grid_result, X_test, y_test, cv=5, n_jobs = 1, verbose=1, scoring='r2')
np.mean(predictions)

In [ ]:
print("R^2: %.3f (%.3f)" % (predictions.mean(), predictions.std()))

In [ ]:
grid_result.best_estimator_.model

In [ ]:
grid_result.best_params_

In [ ]:
# serialize model to JSON
model_json = grid_result.best_estimator_.model.to_json()
with open("model_nn_noespacial.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
grid_result.best_estimator_.model.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
model = Sequential()
model.add(Dense(20, input_dim=X_train.shape[1], kernel_initializer='uniform', activation='linear', kernel_constraint=maxnorm(1)))
model.add(Dropout(0.06))
model.add(Dense(20, input_dim=X_train.shape[1], kernel_initializer='uniform', activation='linear', kernel_constraint=maxnorm(1)))
model.add(Dropout(0.06))
model.add(Dense(20, input_dim=X_train.shape[1], kernel_initializer='uniform', activation='linear', kernel_constraint=maxnorm(1)))
model.add(Dropout(0.06))
model.add(Dense(1, kernel_initializer='uniform', activation='linear'))
# Compile model
model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.01, momentum=0.05), metrics=['mean_absolute_error'])
model.fit(X_train, y_train, epochs=30, batch_size=15)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
pred_NN = model.predict(X_test)
pred_NN = pd.DataFrame(pred_NN)
pred_NN

In [ ]:
mean_squared_error(y_test,pred_NN)

In [ ]:
mean_absolute_error(y_test,pred_NN)

In [ ]:
r2_score(y_test,pred_NN)

In [ ]:
# This returns a multi-layer-perceptron model in Keras.
def get_keras_model(num_hidden_layers, num_neurons_per_layer, dropout_rate, activation):
    # create the MLP model.
    # define the layers.
    inputs = tf.keras.Input(shape=(X_train.shape[1],))  # input layer.
    x = layers.Dropout(dropout_rate)(inputs) # dropout on the weights.
    # Add the hidden layers.
    for i in range(num_hidden_layers):
        x = layers.Dense(num_neurons_per_layer, activation=activation)(x)
        x = layers.Dropout(dropout_rate)(x)
    # output layer.
    outputs = layers.Dense(1, activation='linear')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
# This function takes in the hyperparameters and returns a score (Cross validation).
def keras_mlp_cv_score(parameterization, weight=None):
    model = get_keras_model(parameterization.get('num_hidden_layers'), parameterization.get('neurons_per_layer'),
                            parameterization.get('dropout_rate'), parameterization.get('activation'))    
    opt = parameterization.get('optimizer')
    opt = opt.lower()
    learning_rate = parameterization.get('learning_rate')
    if opt == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif opt == 'rms':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    NUM_EPOCHS = 50
    # Specify the training configuration.
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse', 'mae'])
    data = X_train
    labels = y_train
    # fit the model using validation_data=(X_val,y_val) validation set.
    res = model.fit(data, labels, epochs=NUM_EPOCHS, batch_size=parameterization.get('batch_size'), validation_data=(X_test,y_test))
    # look at the last 10 epochs. Get the mean and standard deviation of the validation score.
    last10_scores = np.array(res.history['val_loss'][-10:])
    mean = last10_scores.mean()
    sem = last10_scores.std()
    # If the model didn't converge then set a high loss.
    if np.isnan(mean):
        return 9999.0, 0.0    
    return mean, sem

In [ ]:
# Define the search space.
parameters=[
    {
        "name": "learning_rate",
        "type": "range",
        "bounds": [0.001, 0.1],
        "log_scale": True,
    },
    {
        "name": "dropout_rate",
        "type": "range",
        "bounds": [0.01, 0.2],
        "log_scale": True,
    },
    {
        "name": "num_hidden_layers",
        "type": "range",
        "bounds": [1, 4],
        "value_type": "int"
    },
    {
        "name": "neurons_per_layer",
        "type": "range",
        "bounds": [10, 50],
        "value_type": "int"
    },
    {
        "name": "batch_size",
        "type": "choice",
        "values": [10, 50],
    },
    
    {
        "name": "activation",
        "type": "choice",
        "values": ['tanh', 'relu', 'linear'],
    },
    {
        "name": "optimizer",
        "type": "choice",
        "values": ['rms', 'SGD', 'adam'],
    },
]

In [ ]:
init_notebook_plotting()
ax_client = AxClient()
# create the experiment.
ax_client.create_experiment(
    name="keras_experiment",
    parameters=parameters,
    objective_name='keras_cv',
    minimize=True)
def evaluate(parameters):
    return {"keras_cv": keras_mlp_cv_score(parameters)}

In [ ]:
for i in range(100):
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=evaluate(parameters))

In [ ]:
resultados = ax_client.get_trials_data_frame().sort_values('trial_index')
resultados

In [ ]:
ax_client.save

In [ ]:
best_parameters, values = ax_client.get_best_parameters()
# the best set of parameters.
for k in best_parameters.items():
    print(k)
print()
# the best score achieved.
means, covariances = values
print(means)

In [ ]:
# MEjor red neuronal con datos de no pago abril incluidos como cero (0)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(50, input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.04284164227732299),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.04284164227732299),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.04284164227732299),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.04284164227732299),
    tf.keras.layers.Dense(1,activation='linear')
])
model.compile(loss='mse', optimizer=tf.keras.optimizers.SGD(learning_rate = 0.05563670130830676), metrics=['mse', 'mae'])
model.fit(X_train, y_train, epochs=50, batch_size=10) #validation_data=(X_val,y_val)
model.evaluate(X_test, y_test)

In [ ]:
pred_NN2 = model.predict(X_test)
pred_NN2 = pd.DataFrame(pred_NN2)
pred_NN2

In [ ]:
mean_squared_error(y_test,pred_NN2)

In [ ]:
mean_absolute_error(y_test,pred_NN2)

In [ ]:
r2_score(y_test,pred_NN2)

## XGboost

In [ ]:
from sklearn.metrics import r2_score
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, tpe

#Bayesian Optimization implementation
regressor = xgb.XGBRegressor()

## Hyper Parameter Optimization
gamma = [1,2,3,4,5,6,7,8,9,10,13,16,20,25,30,35,40,50] #
n_estimators = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 130, 150, 170, 190, 200, 220, 240, 260, 280, 300] #
max_depth = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 140, 160, 180, 200] #
booster=['gblinear'] #,'' 
learning_rate=[0.0001, 0.001, 0.01, 0.05, 0.1, 0.20, 0.25, 0.3, 0.4] #
min_child_weight=[0,0.5,1,2,3,4,5,6,7,8,9,10] #
base_score=[0.25,0.5,0.75,1,1.5,2] #

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'gamma': gamma,
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
    }

# Set up the random search with 4-fold cross validation
random_cv = RandomizedSearchCV(estimator = regressor,
            param_distributions = hyperparameter_grid,
            cv = 5, n_iter = 50,
            scoring = 'r2',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state = 42)

random_cv.fit(X_train,y_train)
random_cv.best_estimator_

In [ ]:
score_xgb = random_cv.score(X_test, y_test)
print(score_xgb)

In [ ]:
pred_xgb = random_cv.predict(X_test)
np.mean(pred_xgb)

In [ ]:
mean_squared_error(y_test,pred_xgb)

In [ ]:
mean_absolute_error(y_test,pred_xgb)

In [ ]:
r2_score(y_test,pred_xgb)

## Support Vector Machine

In [ ]:
svr = SVR(C=20, gamma=1e-06, kernel='linear')
svr.fit(X_train, y_train)

In [ ]:
score_svr = svr.score(X_test, y_test)
print(score_svr)

In [ ]:
pred_svr = svr.predict(X_test)
np.mean(pred_svr)

In [ ]:
mean_squared_error(y_test,pred_svr)

In [ ]:
mean_absolute_error(y_test,pred_svr)

In [ ]:
r2_score(y_test,pred_svr)

In [ ]:
from sklearn.model_selection import StratifiedKFold
# Tuning of parameters for regression by cross-validation
# Parameters for tuning
parameters = {'kernel': ['linear'], 
              'gamma': [1e-06],
              'C': [20]}
# Set up the k-fold cross-validation
#kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
print("Tuning hyper-parameters")
svr = SVR()
svr = GridSearchCV(estimator = svr, param_grid = parameters, cv = 5, n_jobs = 1, verbose = 2, scoring='r2')
svr.fit(X_train, y_train)

# Checking the score for all parameters
print("Grid scores on training set:")
means = svr.cv_results_['mean_test_score']
stds = svr.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, svr.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))